## RAG from Scratch 

In [55]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# MODEL = "gpt-3.5-turbo"
MODEL = "llama3"

In [56]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.llms import Ollama
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

In [58]:
if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key= OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)
    print(embeddings)

# model.invoke("Tell me a Joke")

base_url='http://localhost:11434' model='llama3' embed_instruction='passage: ' query_instruction='query: ' mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None show_progress=False headers=None model_kwargs=None


In [26]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?"

In [45]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("sample.pdf")
pages = loader.load_and_split()
pages

[Document(page_content="ABOUT  US:\nWe've Been Thriving in 10 Y ears\nThe Tech Ar ea\nWe are a pioneer of chatbots in Nepal . Our hist ory dates back t o 2011 when we star ted as an outsour cing company catering t o various clients acr oss US\nand E urope .\xa0\nWith the adv ent of chatbots acr oss the globe, we r ealiz ed the potential and v alue it can pr ovide t o the business and intr oduced chatbot in Nepal in 2018.\xa0\nWe are a team of distinguished pr ofessionals with se veral years of experience working for se veral clients acr oss the globe .\nOur aim is t o intr oduce chatbots acr oss e very business in Nepal and enhance the cust omer experience .\nOur aim is t o intr oduce chatbots acr oss e very business in Nepal and enhan\nOur Hist ory\nWe are a pioneer of chatbots in Nepal . Our history dates back to 2\noutsour cing company catering t o various clients acr oss US and E urope .\nOur Mission\nTo enhance the experience of cust omers thus leading t o overall business gr\nWho

In [44]:
from langchain.prompts import PromptTemplate

template = """ 
Answer the question based on the context below. If the context is not relevant, just reply "I don't know"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate(template=template)
print(prompt.format(context = "Here is some context", question = "Here is a question"))

 
Answer the question based on the context below. If the context is not relevant, just reply "I don't know"

Context: Here is some context

Question: Here is a question



In [46]:
llm_chain = prompt | model | parser

In [47]:
llm_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [48]:
llm_chain.invoke(
    {"context": "The name of the company is BLESSED ENGINEERING.",
    "question": "what's the name of company?"
    }
)

'The answer is: Blessed Engineering!'

### Store the Document in DocArrayInMemoryStore. But in real applications, we will be storing it vector database.

In [59]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [63]:
retriever = vectorstore.as_retriever()

retriever.invoke("experiences")

[Document(page_content='Automated Reminders\nWith tr ansaction emails , aut omatic email and noti\x00cations y our cust omers ar e mor e engaged and r ealiz e an\nenhanced cust omer experience.\nDashboard\nYour e very inter action ar e captur ed and passed thr ough an analytics engine t o provide y ou with engr ossing and\nhighly useful r epor ts and v alues for or ganization thr ough cust om dashboar d.\n(/)\uf00d\n6/27/24, 1:14 PM About Us - Palm Mind\nhttps://palmmind.com/about-us/ 3/5', metadata={'source': 'sample.pdf', 'page': 2}),
 Document(page_content="ABOUT  US:\nWe've Been Thriving in 10 Y ears\nThe Tech Ar ea\nWe are a pioneer of chatbots in Nepal . Our hist ory dates back t o 2011 when we star ted as an outsour cing company catering t o various clients acr oss US\nand E urope .\xa0\nWith the adv ent of chatbots acr oss the globe, we r ealiz ed the potential and v alue it can pr ovide t o the business and intr oduced chatbot in Nepal in 2018.\xa0\nWe are a team of distinguis

In [67]:
from operator import itemgetter
chain = ( 
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }

    |prompt | model | parser)

chain.invoke({"question": "what is our history"})

'Our history dates back to 2011 when we started as an outsourcing company catering to various clients across US and Europe, and in 2018, we introduced chatbots in Nepal.'

In [68]:
questions = [
    "What is the purpose of this company?",
    "when was this company built?",
    "what are your products?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of this company?
Answer: Based on the context, it seems that the main purpose of this company, Palm Mind, is to provide chatbot solutions and services. They mention being a "pioneer of chatbots in Nepal" and having a history dating back to 2011 when they started as an outsourcing company catering to various clients across the US and Europe. They also mention introducing chatbots in Nepal in 2018.

Additionally, they highlight their capabilities in areas such as:

* Machine Learning
* Automated Reminders
* Customer Churn Prediction
* Fraud Detection
* HR Analytics
* Preventive Maintenance
* Social Media Monitoring

Overall, it appears that Palm Mind's primary goal is to help businesses enhance their customer experience through the use of chatbots and related technologies.

Question: when was this company built?
Answer: According to the context, the company was started as an outsourcing company in 2011. Later, they introduced chatbots in Nepal in 2018.

Ques

In [70]:
chain.batch([{"question": q} for q in questions])

['Based on the provided context, it appears that the primary purpose of Palm Mind Computer Consultancy Pvt. Ltd. is to provide chatbot solutions and services to businesses, particularly in Nepal. They have been successful in introducing chatbots across various sectors in Nepal and aim to enhance customer experiences through their technology.',
 'According to the context, the company Palm Mind started as an outsourcing company in 2011 and introduced chatbots in Nepal in 2018. However, it\'s not explicitly stated when the company was "built" or incorporated.',
 'Based on the context, it appears that Palm Mind offers a range of products and services, including:\n\n* Highly-tailored product recommendations\n* Customer Churn Prediction\n* Fraud Detection\n* HR Analytics\n* Psychological Scoring\n* Preventive Maintenance\n* Social Media Monitoring\n* Chatbot solutions with machine learning capabilities (specifically mentioning English and Roman Nepali conversations)\n\nNote that these produc